In [ ]:
"""This is a minimal example of using Tianshou with MARL to train agents.

Author: Will (https://github.com/WillDudley)

Python version used: 3.8.10

Requirements:
pettingzoo == 1.22.0
git+https://github.com/thu-ml/tianshou
"""

import os
from typing import Optional, Tuple

import gymnasium as gym
import numpy as np
import torch
from tianshou.data import Collector, VectorReplayBuffer
from tianshou.env import DummyVectorEnv
from tianshou.env.pettingzoo_env import PettingZooEnv
from tianshou.policy import BasePolicy, DQNPolicy, MultiAgentPolicyManager, RandomPolicy
from tianshou.trainer import offpolicy_trainer
from tianshou.utils.net.common import Net

from pettingzoo.classic import tictactoe_v3


def _get_agents(
    agent_learn: Optional[BasePolicy] = None,
    agent_opponent: Optional[BasePolicy] = None,
    optim: Optional[torch.optim.Optimizer] = None,
) -> Tuple[BasePolicy, torch.optim.Optimizer, list]:
    env = _get_env()
    observation_space = (
        env.observation_space["observation"]
        if isinstance(env.observation_space, gym.spaces.Dict)
        else env.observation_space
    )
    if agent_learn is None:
        # model
        net = Net(
            state_shape=observation_space.shape or observation_space.n,
            action_shape=env.action_space.shape or env.action_space.n,
            hidden_sizes=[128, 128, 128, 128],
            device="cuda" if torch.cuda.is_available() else "cpu",
        ).to("cuda" if torch.cuda.is_available() else "cpu")
        if optim is None:
            optim = torch.optim.Adam(net.parameters(), lr=1e-4)
        agent_learn = DQNPolicy(
            model=net,
            optim=optim,
            discount_factor=0.9,
            estimation_step=3,
            target_update_freq=320,
        )

    if agent_opponent is None:
        agent_opponent = RandomPolicy()

    agents = [agent_opponent, agent_learn]
    policy = MultiAgentPolicyManager(agents, env)
    return policy, optim, env.agents


def _get_env():
    """This function is needed to provide callables for DummyVectorEnv."""
    return PettingZooEnv(tictactoe_v3.env())


if __name__ == "__main__":
    # ======== Step 1: Environment setup =========
    train_envs = DummyVectorEnv([_get_env for _ in range(10)])
    test_envs = DummyVectorEnv([_get_env for _ in range(10)])

    # seed
    seed = 1
    np.random.seed(seed)
    torch.manual_seed(seed)
    train_envs.seed(seed)
    test_envs.seed(seed)

    # ======== Step 2: Agent setup =========
    policy, optim, agents = _get_agents()

    # ======== Step 3: Collector setup =========
    train_collector = Collector(
        policy,
        train_envs,
        VectorReplayBuffer(20_000, len(train_envs)),
        exploration_noise=True,
    )
    test_collector = Collector(policy, test_envs, exploration_noise=True)
    # policy.set_eps(1)
    train_collector.collect(n_step=64 * 10)  # batch size * training_num

    # ======== Step 4: Callback functions setup =========
    def save_best_fn(policy):
        model_save_path = os.path.join("log", "rps", "dqn", "policy.pth")
        os.makedirs(os.path.join("log", "rps", "dqn"), exist_ok=True)
        torch.save(policy.policies[agents[1]].state_dict(), model_save_path)

    def stop_fn(mean_rewards):
        return mean_rewards >= 0.6

    def train_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.1)

    def test_fn(epoch, env_step):
        policy.policies[agents[1]].set_eps(0.05)

    def reward_metric(rews):
        return rews[:, 1]

    # ======== Step 5: Run the trainer =========
    result = offpolicy_trainer(
        policy=policy,
        train_collector=train_collector,
        test_collector=test_collector,
        max_epoch=50,
        step_per_epoch=1000,
        step_per_collect=50,
        episode_per_test=10,
        batch_size=64,
        train_fn=train_fn,
        test_fn=test_fn,
        stop_fn=stop_fn,
        save_best_fn=save_best_fn,
        update_per_step=0.1,
        test_in_train=False,
        reward_metric=reward_metric,
    )

    # return result, policy.policies[agents[1]]
    print(f"\n==========Result==========\n{result}")
    print("\n(the trained policy can be accessed via policy.policies[agents[1]])")

In [ ]:
performance_benchmark(env)

In [ ]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
from pettingzoo.test import performance_benchmark
import cProfile
cProfile.run("performance_benchmark(env)")

In [ ]:
%conda install scipy

In [6]:
from pettingzoo.utils.wrappers import BaseWrapper
from gymnasium.wrappers import FlattenObservation
import Environment
from tianshou.env import PettingZooEnv
env = Environment.VehicleJobSchedulingEnvACE()


env = BaseWrapper(env)

env = PettingZooEnv(env)



In [9]:
obs, _, _, _,_= env.step(1)

In [15]:
from gym.spaces import flatten,flatten_space

In [16]:
flatten_space(env.observation_space)

NotImplementedError: Unknown space: `Dict('avail_slot': Box(0, 100, (10, 2), int8), 'request_job': Dict('res_vec': MultiDiscrete([20 30]), 'len': Discrete(10), 'priority': Discrete(6)))`

In [13]:
flatten(env.observation_space, obs)

NotImplementedError: Unknown space: `Dict('avail_slot': Box(0, 100, (10, 2), int8), 'request_job': Dict('res_vec': MultiDiscrete([20 30]), 'len': Discrete(10), 'priority': Discrete(6)))`

In [28]:
obs = env.observation_space


In [29]:
obs

Box(0, [100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1], (86,), int64)

In [30]:
obss = obs.sample()

In [31]:
obss

array([61, 70, 29,  8, 60, 72, 96, 23, 58, 32, 37, 97,  5, 27, 26, 90,  6,
       61, 17, 38,  0,  0,  1,  1,  1,  0,  0,  1,  0,  1,  0,  0,  1,  1,
        1,  0,  0,  1,  0,  1,  1,  1,  1,  0,  0,  1,  1,  1,  1,  1,  0,
        0,  1,  1,  0,  0,  1,  0,  1,  0,  0,  1,  1,  1,  1,  1,  0,  1,
        0,  1,  0,  0,  0,  1,  1,  1,  0,  1,  0,  0,  1,  0,  1,  1,  0,
        1])

In [11]:
import numpy as np

In [25]:
np.concatenate([np.ravel(obss["avail_slot"]),np.hstack([obss["request_job"][item] for item in obss["request_job"]])])

array([100,  36,  62,  95,  77,  33,  43,  89,  70,  85,  56,  80,  29,
        67,   3,   0,  15,   4,  59,  92,   4,  25,   7,   0])

In [15]:
[obss["request_job"][item] for item in obss["request_job"]]

[array([ 4, 25]), 7, 0]

In [20]:
np.concatenate(([[ 4, 25], 7, 0]),axis=0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 0 dimension(s)

In [23]:
np.hstack([obss["request_job"][item] for item in obss["request_job"]])

array([ 4, 25,  7,  0])

In [12]:
flatten_obs = np.ravel(obss["avail_slot"])


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 0 dimension(s)

In [8]:
def flaten_obs(obss):
    return obss.flatten()

In [3]:
import gym.spaces as spaces
from gym.spaces.utils import flatten, flatten_space
import numpy as np
from collections import OrderedDict
import supersuit
nobs = spaces.Dict(
            OrderedDict(
                [
                    (
                        "avail_slot",
                        spaces.Box(
                            low=0,
                            high=100,
                            shape=(
                                10,
                                2,
                            ),
                            dtype=np.int8,
                        ),
                    ),
                    (
                        "request_res_vec",
                        spaces.MultiDiscrete([20,30]),
                    ),
                    (
                        "request_len",
                        spaces.Discrete(10),
                    ),
                    (
                        "request_priority",
                        spaces.Discrete(6),
                    ),
                ]
            )
        )
obss = nobs.sample()
obs_flat = flatten_space(nobs)
obssf = flatten(nobs,obss)


In [4]:
nobs

Dict(avail_slot:Box(0, 100, (10, 2), int8), request_res_vec:MultiDiscrete([20 30]), request_len:Discrete(10), request_priority:Discrete(6))

In [9]:
obs

Dict('avail_slot': Box(0, 100, (10, 2), int8), 'request_res_vec': MultiDiscrete([20 30]), 'request_len': Discrete(10), 'request_priority': Discrete(6))

In [3]:
obssf


array([42, 95, 51, 26, 96, 45, 83, 96, 87, 82, 88, 76, 79,  4, 63, 52,  7,
       93, 15,  6,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0])

In [1]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
from gymnasium import spaces
import gymnasium.spaces.utils as utils
import numpy as np
from collections import OrderedDict

utils.flatten(env.obs,env.parameters.cluster.machines[0].observe())

array([20, 44, 20, 44, 20, 44, 20, 44, 20, 44, 20, 44, 20, 44, 20, 44, 20,
       44, 20, 44,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,
        0])

In [4]:
env.parameters.cluster.machines[0].observe()

OrderedDict([('avail_slot',
              array([[23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40],
                     [23, 40]], dtype=int8)),
             ('request_res_vec', [0, 0]),
             ('request_len', 0),
             ('request_priority', 0)])

In [6]:
flatten(env.observation_space, env.obs)

NameError: name 'flatten' is not defined

In [5]:
env.obs.sample()

OrderedDict([('avail_slot',
              array([[26, 56],
                     [47,  6],
                     [31, 70],
                     [97, 54],
                     [ 5, 23],
                     [73, 15],
                     [ 8, 96],
                     [42, 76],
                     [81, 33],
                     [22, 12]], dtype=int8)),
             ('request_res_vec', array([ 8, 19])),
             ('request_len', 3),
             ('request_priority', 3)])

In [14]:
env.obs

Dict('avail_slot': Box(0, 100, (10, 2), int8), 'request_res_vec': MultiDiscrete([20 30]), 'request_len': Discrete(10), 'request_priority': Discrete(6))

In [15]:
obs

Dict('avail_slot': Box(0, 100, (10, 2), int8), 'request_res_vec': MultiDiscrete([20 30]), 'request_len': Discrete(10), 'request_priority': Discrete(6))

In [17]:
obs_flat = utils.flatten_space(env.obs)
obs_flat

Box(0, [100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100 100
 100 100   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1], (86,), int64)

In [19]:
obss = obs.sample()
obs_flat = utils.flatten_space(obs)
obssf = utils.flatten(obs,obss)

In [3]:
np.max([20,30])

30

In [1]:
import Environment
env = Environment.VehicleJobSchedulingEnvACE()
import cProfile
from pettingzoo.test import performance_benchmark
cProfile.run('performance_benchmark(env)')
     

Starting performance benchmark
5449.794705495007 turns per second
544.9794705495008 cycles per second
Finished performance benchmark
         3299429 function calls (3190399 primitive calls) in 5.002 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    55642    0.056    0.000    0.495    0.000 <__array_function__ internals>:177(all)
       20    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amax)
       40    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(amin)
    27250    0.022    0.000    0.230    0.000 <__array_function__ internals>:177(any)
     1887    0.002    0.000    0.025    0.000 <__array_function__ internals>:177(argsort)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(atleast_1d)
        3    0.000    0.000    0.000    0.000 <__array_function__ internals>:177(broadcast_arrays)
    27264    0.023    0.000    0.121    0.000 <__array_functi